In [13]:
import warnings
from tokenize import endpats

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from statannotations.Annotator import Annotator
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

In [14]:
df = pd.read_csv('source/FReDA4.csv')

In [15]:
def val_to_category(val):
    if val <= 0.3:
        return 0
    elif 0.3 < val <= 1.5:
        return 1
    elif 1.5 < val <= 2.5:
        return 2
    elif 2.5 < val <= 3.5:
        return 3
    elif 3.5 < val <= 4.5:
        return 4
    elif 4.5 < val:
        return 5
    return None

In [23]:
# Satisfied
df.loc[df['Group1'] == 'SubGroup1', 'Group4'] = 'Satisfied'

# Deprived groups
df.loc[df['Group1'] == 'SubGroup3', 'Group4'] = 'Deprived_Couples'
# df.loc[df['Group2'] == 'One-sided Deprivation', 'Group4'] = 'Deprived_One'
df.loc[df['Group1'] == 'SubGroup2', 'Group4'] = 'Deprived_Me'
df.loc[df['Group1'] == 'SubGroup7', 'Group4'] = 'Deprived_Partner'

# Saturated groups
df.loc[df['Group1'] == 'SubGroup6', 'Group4'] = 'Saturated_Couples'
# df.loc[df['Group2'] == 'One-sided Saturation', 'Group4'] = 'Saturated_One'
df.loc[df['Group1'] == 'SubGroup5', 'Group4'] = 'Saturated_Me'
df.loc[df['Group1'] == 'SubGroup8', 'Group4'] = 'Saturated_Partner'

# Mixed groups
df.loc[df['Group1'] == 'SubGroup4', 'Group4'] = 'Mixed_Couples'
df.loc[df['Group1'] == 'SubGroup9', 'Group4'] = 'Mixed_Couples'

In [ ]:
df['Freq'] = df['Frequency'].apply(val_to_category)
df['Des'] = df['Desire'].apply(val_to_category)

df.groupby(['Group4'])[['Des']].value_counts()

In [24]:
# TODO:
# 1. COUNT TOTAL FOR EACH GROUP
# 2. CALCULATE %
# 3. PLOT FREQUENCY AND DESIRE PERO GROUP (9)

Group4             Des
Deprived_Couples   3      1186
                   2       849
                   4       809
                   5       296
                   1       262
Deprived_Me        3       647
                   4       521
                   2       426
                   5       260
                   1       216
                   0        14
Deprived_Partner   3       843
                   4       767
                   2       615
                   5       316
                   1       302
                   0        13
Mixed_Couples      3       194
                   2       144
                   1       125
                   4       106
                   5        53
                   0        38
Satisfied          4      1200
                   3       996
                   5       806
                   2       557
                   1       269
                   0        14
Saturated_Couples  1        25
                   3        21
                